In [1]:
!pip install vaderSentiment

In [2]:
import pandas as pd

In [3]:
import string

In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
import pandas as pd
data = pd.read_csv("news_data_aapl.csv")
data = data.dropna()
print(data.head())

   Unnamed: 0   TimeStamp                                           Headline
0           0  2022-06-23  Swiss detail revenue split for global minimum ...
1           1  2022-06-22  Meta and other tech giants form metaverse stan...
2           2  2022-06-22  Shopify unveils new tools, Twitter tie-up to b...
3           3  2022-06-22  Volkswagen CEO questions whether Apple plans t...
4           4  2022-06-21         Wall Street gains over 2% in broad rebound


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Tanmay'PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
from nltk.corpus import stopwords

In [6]:
data.drop(data.filter(regex="Unnamed"),axis=1, inplace=True)

In [7]:
data[["Headline"]]
data["Headline"] = data["Headline"].astype(str)
data.head()

,TimeStamp,Headline
0,2022-06-23,Swiss detail revenue split for global minimum ...
1,2022-06-22,Meta and other tech giants form metaverse stan...
2,2022-06-22,"Shopify unveils new tools, Twitter tie-up to b..."
3,2022-06-22,Volkswagen CEO questions whether Apple plans t...
4,2022-06-21,Wall Street gains over 2% in broad rebound


In [8]:
data["Headline"] = data["Headline"].str.lower()
data.head()

,TimeStamp,Headline
0,2022-06-23,swiss detail revenue split for global minimum ...
1,2022-06-22,meta and other tech giants form metaverse stan...
2,2022-06-22,"shopify unveils new tools, twitter tie-up to b..."
3,2022-06-22,volkswagen ceo questions whether apple plans t...
4,2022-06-21,wall street gains over 2% in broad rebound


In [9]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data["Headline"] = data["Headline"].apply(lambda text: remove_punctuation(text))
data.head()

,TimeStamp,Headline
0,2022-06-23,swiss detail revenue split for global minimum ...
1,2022-06-22,meta and other tech giants form metaverse stan...
2,2022-06-22,shopify unveils new tools twitter tieup to bea...
3,2022-06-22,volkswagen ceo questions whether apple plans t...
4,2022-06-21,wall street gains over 2 in broad rebound


In [10]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [11]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data["Headline"] = data["Headline"].apply(lambda text: remove_stopwords(text))
data.head()

,TimeStamp,Headline
0,2022-06-23,swiss detail revenue split global minimum tax ...
1,2022-06-22,meta tech giants form metaverse standards body...
2,2022-06-22,shopify unveils new tools twitter tieup beat e...
3,2022-06-22,volkswagen ceo questions whether apple plans p...
4,2022-06-21,wall street gains 2 broad rebound


In [12]:
from nltk.stem.porter import PorterStemmer

In [13]:
""" stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

data["Headline"] = data["Headline"].apply(lambda text: stem_words(text))
data.head() """

' stemmer = PorterStemmer()\ndef stem_words(text):\n    return " ".join([stemmer.stem(word) for word in text.split()])\n\ndata["Headline"] = data["Headline"].apply(lambda text: stem_words(text))\ndata.head() '

In [14]:
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Headline"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Headline"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Headline"]]
data['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data["Headline"]]
data.head()

,TimeStamp,Headline,Positive,Negative,Neutral,Compound
0,2022-06-23,swiss detail revenue split global minimum tax ...,0.000,0.0,1.000,0.00
1,2022-06-22,meta tech giants form metaverse standards body...,0.000,0.0,1.000,0.00
2,2022-06-22,shopify unveils new tools twitter tieup beat e...,0.000,0.0,1.000,0.00
3,2022-06-22,volkswagen ceo questions whether apple plans p...,0.000,0.0,1.000,0.00
4,2022-06-21,wall street gains 2 broad rebound,0.375,0.0,0.625,0.34


In [15]:
score = data["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append(1)
    elif i <= -0.05 :
        sentiment.append(-1)
    else:
        sentiment.append(0)
data["Sentiment"] = sentiment
data.head()

,TimeStamp,Headline,Positive,Negative,Neutral,Compound,Sentiment
0,2022-06-23,swiss detail revenue split global minimum tax ...,0.000,0.0,1.000,0.00,0
1,2022-06-22,meta tech giants form metaverse standards body...,0.000,0.0,1.000,0.00,0
2,2022-06-22,shopify unveils new tools twitter tieup beat e...,0.000,0.0,1.000,0.00,0
3,2022-06-22,volkswagen ceo questions whether apple plans p...,0.000,0.0,1.000,0.00,0
4,2022-06-21,wall street gains 2 broad rebound,0.375,0.0,0.625,0.34,1


In [16]:
print(data["Sentiment"].value_counts())

 0    11277
 1     8584
-1     5055
Name: Sentiment, dtype: int64


In [17]:
print(data.groupby(['Sentiment']).count())

           TimeStamp  Headline  Positive  Negative  Neutral  Compound
Sentiment                                                            
-1              5055      5055      5055      5055     5055      5055
 0             11277     11277     11277     11277    11277     11277
 1              8584      8584      8584      8584     8584      8584


In [18]:
data.Sentiment.value_counts()/len(data)

 0    0.452601
 1    0.344518
-1    0.202882
Name: Sentiment, dtype: float64

In [19]:
data[data.Sentiment==0]

,TimeStamp,Headline,Positive,Negative,Neutral,Compound,Sentiment
0,2022-06-23,swiss detail revenue split global minimum tax ...,0.0,0.0,1.0,0.0,0
1,2022-06-22,meta tech giants form metaverse standards body...,0.0,0.0,1.0,0.0,0
2,2022-06-22,shopify unveils new tools twitter tieup beat e...,0.0,0.0,1.0,0.0,0
3,2022-06-22,volkswagen ceo questions whether apple plans p...,0.0,0.0,1.0,0.0,0
5,2022-06-21,meta tech giants form metaverse standards body...,0.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...
24911,2012-07-16,waiting direction markets,0.0,0.0,1.0,0.0,0
24912,2012-07-19,mid year update u canadian stock markets sector,0.0,0.0,1.0,0.0,0
24913,2012-07-23,summer heat scorches europe u,0.0,0.0,1.0,0.0,0
24914,2012-07-23,apple earnings preview quarterly dip deck,0.0,0.0,1.0,0.0,0


In [20]:
data.title = data.title.apply(lambda x: text_to_word_list(x, unidecode))

AttributeError: 'DataFrame' object has no attribute 'title'